<a href="https://colab.research.google.com/github/retnuh-k/peptide-mutation-finder/blob/main/PeptideMutationFinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mutation to peptide sequence pipeline

This notebook automates the process of figuring out changes in protein sequences as a result of mutations found from genome sequencing. Written by Hunter King (https://github.com/retnuh-k/) under the guidance of Dr. Songato Jia and Yimeng Fang as part of the Jia Lab (http://jia.biology.columbia.edu/).

**Required input files:** \\
*   mutation_comparison_tsv_file - file describing the differences in two vcf files outputted by vcftools --gzdiff function (see http://vcftools.sourceforge.net/man_latest.html)
*   geneome_tsv_file - a tsv file of the organism's feature coordinates in gff3 file format (can create by removing header starting with '#' from gff3 file and saving as tsv)
*   genome_fasta_file - a fasta file with each chromosome sequence on a single line and sequence headers written as ">[chromosome name]"

**Full pipeline steps**
1.   Obtain sequencing data of strain of interest and wild type
2.   Download reference genome fasta file
3.   Run freebayes variant detector (see https://github.com/freebayes/freebayes) on both strain and wild type .bam files against the reference genome to get a vcf (variant call file) for each
4.   Filter each vcf file by quality (recommended between 10 and 30) and (optionally) depth using vcffilter or equivalent (https://github.com/vcflib/vcflib)
5.   gzip filtered vcf files
6.   Compare vcf files using vcftools (http://vcftools.sourceforge.net/man_latest.html), run "vcftools --gzvcf test_strain.vcf.gz --gzdiff wild_time.vcf.gz --diff-site --out comparison_output_file". This step ensures that only mutations not shared with the wild type are analyzed.
7.   Rename comparison output file to have .tsv file extension
8.   Download genome fasta file and reformat per the above specifications
9.   Download genome coordinate gff3 file, remove header and save as .tsv file
10.  Upload required files to notebook, specify filenames in code block below, and run notebook 



In [420]:
import numpy as np
import pandas as pd
import re
try:
  import Bio
except:
  !pip install Biopython
from Bio.Seq import Seq
from google.colab import files

#@title Input filenames, then hit Runtime -> Run All
mutation_comparison_tsv_file = "TT_v_WT_DP10QU20.tsv" #@param {type:"string"}
geneome_tsv_file = "Schizosaccharomyces_pombe_all_chromosomes.tsv" #@param {type:"string"}
genome_fasta_file = "pombe_genome.fasta" #@param {type:"string"}
output_filename = "mutation_results.txt" #@param {type:"string"}

# Load Dataframes
mut_df = pd.read_csv(mutation_comparison_tsv_file,sep='\t',header=0)
mask = mut_df['IN_FILE']=='1'
mut_df_1 = mut_df[mask]
genome_df = pd.read_csv(geneome_tsv_file,
                        sep='\t',
                        names=["chromosome","source","seq_type","start_pos","end_pos","score","strand","frame","id"])

exon_m = genome_df['id'].str.contains("exon")
exon_df = genome_df[exon_m]
for row in exon_df.index:
  if "RNA" in str(exon_df.loc[row,"id"]):
    exon_df.drop(row,inplace=True)

mut_df_1.insert(mut_df_1.shape[1],"gene_id",np.empty(mut_df_1.shape[0]))
mut_df_1.insert(mut_df_1.shape[1],"gene_start",np.empty(mut_df_1.shape[0]))
mut_df_1.insert(mut_df_1.shape[1],"gene_end",np.empty(mut_df_1.shape[0]))
mut_df_1.insert(mut_df_1.shape[1],"strand",np.empty(mut_df_1.shape[0]))

chromosome_dfs = {}
for chrom in set(exon_df['chromosome']):
  chrom_mask = exon_df['chromosome'] == chrom
  chromosome_dfs[chrom] = pd.DataFrame(exon_df[chrom_mask])

genome_str = ""
with open(genome_fasta_file, 'r') as file:
  for line in file:
    if '>' in line:
      genome_str = genome_str + "\n" + line
    else:
      genome_str = genome_str + line[:-1]
genome_str = genome_str[1:] # remove first newline

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [421]:
#@title Function definitions

# Store which genes the mutations are in if they are in exons
def binary_mutation_search(mut_df, chrom_df_arr):
  hits = 0
  df = mut_df.copy()
  for i in df.index:
    chrom_df = chrom_df_arr[mut_df_1.loc[i,'CHROM']]
    chrom_df.sort_values(by='start_pos',axis=0,inplace=True)
    mut_pos = int(df.loc[i,'POS1'])

    lower = -1
    upper = len(chrom_df.index)
    idx = int(upper/2)
    chrom_idx = chrom_df.index

    # Binary search
    while True:
      idx_old = idx
      gene_start = int(chrom_df.loc[chrom_idx[idx],'start_pos'])
      gene_end = int(chrom_df.loc[chrom_idx[idx],'end_pos'])

      if mut_pos < gene_start:
        # Mutation precedes gene, look in previous half
        upper = idx
        idx -= int((idx - lower) / 2)

      elif mut_pos <= gene_end:
        # Mutation within gene [start, end], store id
        df.loc[i,'gene_id'] = chrom_df.loc[chrom_idx[idx],"id"]
        df.loc[i,'gene_start'] = chrom_df.loc[chrom_idx[idx],"start_pos"]
        df.loc[i,'gene_end'] = chrom_df.loc[chrom_idx[idx],"end_pos"]
        df.loc[i,'strand'] = chrom_df.loc[chrom_idx[idx],"strand"]

        hits += 1
        break

      else:
        # Mutation follows gene, look in next half
        lower = idx
        idx += int((upper - idx) / 2)

      if idx == idx_old: break # Mutation not within any gene

  print("Number of hits:", hits)
  return df


# Extract gene IDs and names with mutations
def get_gene_dict(mut_id_df, gene_df):
  gene_dict = {}
  for gene in mut_id_df['gene_id']:
    match = re.search(r'^ID=[A-Z0-9\.]*',str(gene))
    if match: 
      gene_dict[match[0]] = None

  name_m = gene_df['id'].str.contains("Name=")
  name_df = gene_df[name_m]
  for name in name_df['id']:
    id_match = re.search(r'^ID=[A-Z0-9\.]*',str(name))
    name_match = re.search(r'(Name=)(.*)',str(name))
    if id_match[0] in gene_dict.keys():
      gene_dict[id_match[0]] = name_match[2]
  return gene_dict


# Apply all mutations in the mutation df to a specific gene id
def insert_mutation(mutation_df, gene_id):

  # Filter df by specific gene_id
  mask = mutation_df['gene_id'].str.contains(gene_id)
  df = mutation_df[mask]

  # Define key variables
  first_idx = df.index[0]
  chrom = str(df.loc[first_idx,'CHROM']) + "\n"
  gene_start = int(df.loc[first_idx,'gene_start'])
  gene_end = int(df.loc[first_idx,'gene_end'])
  chrom_start = genome_str.index(chrom) + len(chrom) - 1 # -1 because genome positions index starting with 1

  # Find gene nt sequence in genome string
  gene = genome_str[chrom_start + gene_start : chrom_start + gene_end + 1]
  old_seq = Seq(gene)

  # Apply each mutation to gene nt sequence
  for i in df.index:
    mut_pos = int(df.loc[i,'POS1'])
    mut_gene_pos = mut_pos - gene_start
    gene = gene[:mut_gene_pos] + str(df.loc[i,'ALT1']) + gene[mut_gene_pos + len(str(df.loc[i,'REF1'])):]
  
  # Return new gene sequence (accounting for reverse strand)
  new_seq = Seq(gene)
  if str(df.loc[first_idx,'strand']) == '+':
    return old_seq, new_seq, "+"
  else:
    return old_seq.reverse_complement(), new_seq.reverse_complement(), "-"

In [422]:
#@title Print names of genes with mutations

mut_id_df = binary_mutation_search(mut_df_1, chromosome_dfs)

# Remove non genes and print systematic gene names
gene_dict = get_gene_dict(mut_id_df, genome_df)
print("Systematic gene names:\n",np.array(list(gene_dict.keys())))
print("\nNumber of genes with mutations:\n",len(gene_dict.keys()))
print("\nKnown gene names (if in genome dataframe):\n",np.array(list(set(gene_dict.values()))))

Number of hits: 70
Systematic gene names:
 ['ID=SPAC212.11.1' 'ID=SPAC977.01.1' 'ID=SPAC977.09' 'ID=SPAC24H6.06.1'
 'ID=SPAC30D11.04' 'ID=SPAC3C7.01' 'ID=SPAC27F1.03' 'ID=SPAC4D7.09.1'
 'ID=SPAC750.08' 'ID=SPBC1683.07.1' 'ID=SPBC1685.16.1' 'ID=SPBPJ4664.02.1'
 'ID=SPBC409.11.1' 'ID=SPBC1E8.04.1' 'ID=SPBC1A4.06' 'ID=SPBC1711.13.1'
 'ID=SPBC14C8.09' 'ID=SPBC336.04.1' 'ID=SPBC336.13' 'ID=SPBC16D10.07'
 'ID=SPBCPT2R1.08' 'ID=SPCC548.03' 'ID=SPCC1020.06' 'ID=SPCC63.11.1'
 'ID=SPCC188.09' 'ID=SPCC584.03' 'ID=SPCC1442.03.1' 'ID=SPCC737.08.1']

Number of genes with mutations:
 28

Known gene names (if in genome dataframe):
 [None 'sir2' 'tal1' 'uch1' 'wtf4' 'nup124' 'dbl3' 'tlh2' 'tam41' 'plb4'
 'mmp2' 'pfl4' 'sac12']


In [423]:
#@title Display mutation dataframe

#Cut mutation dataframe to only mutations within genes
mut_hit_df = mut_id_df.copy()
for row in mut_hit_df.index:
  if not "ID" in str(mut_hit_df.loc[row,"gene_id"]):
    mut_hit_df.drop(row,inplace=True)
mut_hit_df

,CHROM,POS1,POS2,IN_FILE,REF1,REF2,ALT1,ALT2,gene_id,gene_start,gene_end,strand
1,I,459,.,1,C,.,A,.,ID=SPAC212.11.1:exon:1;Parent=SPAC212.11.1,1.0,5662.0,-
2,I,3172,.,1,A,.,C,.,ID=SPAC212.11.1:exon:1;Parent=SPAC212.11.1,1.0,5662.0,-
3,I,4635,.,1,T,.,G,.,ID=SPAC212.11.1:exon:1;Parent=SPAC212.11.1,1.0,5662.0,-
10,I,29768,.,1,A,.,G,.,ID=SPAC977.01.1:exon:1;Parent=SPAC977.01.1,29764.0,30716.0,+
11,I,29769,.,1,T,.,A,.,ID=SPAC977.01.1:exon:1;Parent=SPAC977.01.1,29764.0,30716.0,+
...,...,...,...,...,...,...,...,...,...,...,...,...
502,III,858080,.,1,G,.,A,.,ID=SPCC63.11.1:exon:1;Parent=SPCC63.11.1,857153.0,859141.0,+
567,III,1494129,.,1,T,.,G,.,ID=SPCC188.09c.1:exon:1;Parent=SPCC188.09c.1,1492938.0,1494767.0,-
570,III,1533938,.,1,G,.,A,.,ID=SPCC584.03c.1:exon:1;Parent=SPCC584.03c.1,1533022.0,1534677.0,-
576,III,1772337,.,1,G,.,A,.,ID=SPCC1442.03.1:exon:2;Parent=SPCC1442.03.1,1772011.0,1772631.0,+


In [425]:
#@title Write results to file
outfile = open(output_filename, "w")
for gene in set(mut_hit_df['gene_id']):
  old, new, strand = insert_mutation(mut_hit_df,gene)
  outfile.write(f"Results for {gene} on {strand}:\n")
  old_pep = old.transcribe().translate()
  new_pep = new.transcribe().translate()
  if old_pep == new_pep:
    outfile.write("Mutation did not cause an amino acid change")
  else:
    outfile.write(f"New peptide sequence:\n{new_pep}\n")

  for i in range(np.min([len(old_pep),len(new_pep)])):
    if old_pep[i] != new_pep[i]:
      outfile.write(f"AA#{i+1} {old_pep[i]} -> {new_pep[i]}, ")
  outfile.write("\n\n")
outfile.close()
files.download(output_filename)

/usr/local/lib/python3.7/dist-packages/Bio/Seq.py:2983: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>